In [11]:
import pulp as plp
from pulp import LpProblem, LpMaximize, LpVariable, lpSum
from domain.entities import DayOfTheWeek, Employee, Patient
import itertools

### Employee Schedules
We limited the schedules from 7:00 to 12:00 for clarity
| Employee 1 | Monday | Tuesday | Wednesday | Thursday | Friday | Saturday | Sunday |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 07:00 - 7:30  | G1 | G2 | -- | G1 | G3 | -- | -- |
| 07:30 - 8:00  | G1 | G2 | -- | G1 | G3 | -- | -- |
| 08:00 - 8:30  | G1 | G2 | -- | G1 | G3 | -- | -- |
| 08:30 - 9:00  | -- | -- | -- | -- | G3 | -- | -- |
| 09:00 - 9:30  | -- | -- | -- | -- | G3 | -- | -- |
| 09:30 - 10:00 | G2 | G3 | -- | G3 | -- | -- | -- |
| 10:00 - 10:30 | G2 | G3 | -- | G3 | -- | -- | -- |
| 10:30 - 11:00 | G2 | G3 | -- | G3 | -- | -- | -- |
| 11:00 - 11:30 | G1 | G3 | -- | G3 | -- | -- | -- |
| 11:30 - 12:00 | G1 | G3 | -- | G3 | -- | -- | -- |

| Employee 2 | Monday | Tuesday | Wednesday | Thursday | Friday | Saturday | Sunday |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 07:00 - 7:30  | -- | -- | G1 | G2 | -- | -- | -- |
| 07:30 - 8:00  | -- | G1 | G3 | G2 | -- | -- | -- |
| 08:00 - 8:30  | -- | G1 | G3 | -- | -- | -- | -- |
| 08:30 - 9:00  | -- | -- | -- | -- | -- | -- | -- |
| 09:00 - 9:30  | -- | -- | -- | -- | -- | -- | -- |
| 09:30 - 10:00 | G3 | G1 | G2 | -- | -- | -- | -- |
| 10:00 - 10:30 | G3 | G1 | G2 | -- | -- | -- | -- |
| 10:30 - 11:00 | G3 | -- | G2 | G1 | -- | -- | -- |
| 11:00 - 11:30 | G3 | -- | G2 | G1 | -- | -- | -- |
| 11:30 - 12:00 | G3 | -- | G2 | G1 | -- | -- | -- |

| Employee 3 | Monday | Tuesday | Wednesday | Thursday | Friday | Saturday | Sunday |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 07:00 - 7:30  | -- | G3 | G2 | -- | G2 | -- | -- |
| 07:30 - 8:00  | -- | G3 | G2 | -- | G2 | -- | -- |
| 08:00 - 8:30  | -- | -- | G2 | -- | G2 | -- | -- |
| 08:30 - 9:00  | -- | -- | -- | -- | -- | -- | -- |
| 09:00 - 9:30  | -- | G1 | -- | -- | G3 | -- | -- |
| 09:30 - 10:00 | -- | -- | G1 | -- | G1 | -- | -- |
| 10:00 - 10:30 | -- | G1 | G1 | -- | G1 | -- | -- |
| 10:30 - 11:00 | -- | G1 | G1 | -- | G1 | -- | -- |
| 11:00 - 11:30 | -- | -- | -- | -- | -- | -- | -- |
| 11:30 - 12:00 | -- | -- | -- | -- | -- | -- | -- |

| Occupation | Monday | Tuesday | Wednesday | Thursday | Friday | Saturday | Sunday |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 07:00 - 7:30  | G1----- | G2----G3 | ---G1-G2 | G1-G2--- | G3----G2 | -- | -- |
| 07:30 - 8:00  | G1----- | G2-G1-G3 | ---G3-G2 | G1-G2--- | G3----G2 | -- | -- |
| 08:00 - 8:30  | G1----- | G2-G1--- | ---G3-G2 | G1------ | G3----G2 | -- | -- |
| 08:30 - 9:00  | ------- | -------- | -------- | -------- | G3------ | -- | -- |
| 09:00 - 9:30  | ------- | ------G1 | -------- | -------- | G3----G3 | -- | -- |
| 09:30 - 10:00 | G2-G3-- | G3-G1--- | ---G2-G1 | G3------ | ------G1 | -- | -- |
| 10:00 - 10:30 | G2-G3-- | G3-G1-G1 | ---G2-G1 | G3------ | ------G1 | -- | -- |
| 10:30 - 11:00 | G2-G3-- | G3----G1 | ---G2-G1 | G3-G1--- | ------G1 | -- | -- |
| 11:00 - 11:30 | G1-G3-- | G3------ | ---G2--- | G3-G1--- | -------- | -- | -- |
| 11:30 - 12:00 | G1-G3-- | G3------ | ---G2--- | G3-G1--- | -------- | -- | -- |





### Patient Schedules
We limited the schedules from 7:00 to 12:00 for clarity
| Patient 1 | Monday | Tuesday | Wednesday | Thursday | Friday | Saturday | Sunday |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 07:00 - 7:30  | -- | -- | -- | -- | -- | -- | -- |
| 07:30 - 8:00  | -- | -- | -- | -- | G3 | -- | -- |
| 08:00 - 8:30  | -- | -- | -- | -- | G3 | -- | -- |
| 08:30 - 9:00  | -- | -- | -- | -- | G3 | -- | -- |
| 09:00 - 9:30  | -- | -- | -- | -- | G3 | -- | -- |
| 09:30 - 10:00 | G2 | -- | -- | -- | -- | -- | -- |
| 10:00 - 10:30 | G2 | -- | -- | -- | -- | -- | -- |
| 10:30 - 11:00 | -- | -- | -- | -- | -- | -- | -- |
| 11:00 - 11:30 | G1 | -- | -- | -- | -- | -- | -- |
| 11:30 - 12:00 | -- | -- | -- | -- | -- | -- | -- |

| Patient 2 | Monday | Tuesday | Wednesday | Thursday | Friday | Saturday | Sunday |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 07:00 - 7:30  | -- | G3 | -- | -- | -- | -- | -- |
| 07:30 - 8:00  | -- | G3 | -- | -- | G3 | -- | -- |
| 08:00 - 8:30  | -- | -- | -- | -- | G3 | -- | -- |
| 08:30 - 9:00  | -- | -- | -- | -- | G3 | -- | -- |
| 09:00 - 9:30  | -- | -- | -- | -- | G3 | -- | -- |
| 09:30 - 10:00 | -- | -- | -- | -- | -- | -- | -- |
| 10:00 - 10:30 | -- | -- | -- | -- | -- | -- | -- |
| 10:30 - 11:00 | -- | -- | -- | -- | -- | -- | -- |
| 11:00 - 11:30 | -- | -- | -- | -- | -- | -- | -- |
| 11:30 - 12:00 | -- | -- | -- | -- | -- | -- | -- |

| Patient 3 | Monday | Tuesday | Wednesday | Thursday | Friday | Saturday | Sunday |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 07:00 - 7:30  | -- | -- | G2 | -- | -- | -- | -- |
| 07:30 - 8:00  | -- | -- | G2 | -- | -- | -- | -- |
| 08:00 - 8:30  | -- | -- | G2 | -- | -- | -- | -- |
| 08:30 - 9:00  | -- | -- | -- | -- | -- | -- | -- |
| 09:00 - 9:30  | -- | -- | -- | -- | -- | -- | -- |
| 09:30 - 10:00 | -- | -- | G2 | -- | -- | -- | -- |
| 10:00 - 10:30 | -- | -- | G2 | G3 | -- | -- | -- |
| 10:30 - 11:00 | -- | -- | G2 | G3 | -- | -- | -- |
| 11:00 - 11:30 | -- | -- | -- | G3 | -- | -- | -- |
| 11:30 - 12:00 | -- | -- | -- | -- | -- | -- | -- |


In [26]:
# define dummy employees
employee1 = Employee(1)
employee2 = Employee(2)
employee3 = Employee(3)
employee4 = Employee(4)
employee5 = Employee(5)
employee6 = Employee(6)
employee7 = Employee(7)
employee8 = Employee(8)
employee9 = Employee(9)
employee10 = Employee(10)

employee1.schedule.add_items([
        (1, DayOfTheWeek.MONDAY, 7,8.5),
        (1, DayOfTheWeek.MONDAY, 9.5,12),
        (1, DayOfTheWeek.MONDAY, 12.5,14),
        (1, DayOfTheWeek.TUESDAY, 7,8.5),
        (1, DayOfTheWeek.TUESDAY, 9.5,12),
        (1, DayOfTheWeek.THURSDAY,7,8.5),
        (1, DayOfTheWeek.THURSDAY,9.5,12),
        (1, DayOfTheWeek.FRIDAY, 7,9.5),
    ])

employee2.schedule.add_items([
        (1, DayOfTheWeek.MONDAY, 9.5,12),
        (1, DayOfTheWeek.TUESDAY, 7.5,8.5),
        (1, DayOfTheWeek.TUESDAY, 9.5,10.5),
        (1, DayOfTheWeek.WEDNESDAY, 7,7.5),
        (1, DayOfTheWeek.WEDNESDAY,7.5,8.5),
        (1, DayOfTheWeek.WEDNESDAY,9.5,12),
        (1, DayOfTheWeek.THURSDAY, 7,8),
        (1, DayOfTheWeek.THURSDAY, 10.5,12)
    ])

employee3.schedule.add_items([
    (1, DayOfTheWeek.TUESDAY,7,8),
    (1, DayOfTheWeek.TUESDAY,9,9.5),
    (1, DayOfTheWeek.TUESDAY,10,11),
    (1, DayOfTheWeek.WEDNESDAY,7,8.5),
    (1, DayOfTheWeek.WEDNESDAY,9.5,11),
    (1, DayOfTheWeek.FRIDAY,7,8.5),
    (1, DayOfTheWeek.FRIDAY,9,9.5),
    (1, DayOfTheWeek.FRIDAY,9.5,11),
])

employee4.schedule.add_items([
    (1, DayOfTheWeek.MONDAY, 7,8.5),
    (1, DayOfTheWeek.MONDAY, 9.5,12),
    (1, DayOfTheWeek.MONDAY, 12.5,14),
    (1, DayOfTheWeek.TUESDAY, 7,8.5),
    (1, DayOfTheWeek.TUESDAY, 9.5,12),
    (1, DayOfTheWeek.THURSDAY,7,8.5),
    (1, DayOfTheWeek.THURSDAY,9.5,12),
    (1, DayOfTheWeek.FRIDAY, 7,9.5),
])

employee5.schedule.add_items([
    (1, DayOfTheWeek.MONDAY, 9.5,12),
    (1, DayOfTheWeek.TUESDAY, 7.5,8.5),
    (1, DayOfTheWeek.TUESDAY, 9.5,10.5),
    (1, DayOfTheWeek.WEDNESDAY, 7,7.5),
    (1, DayOfTheWeek.WEDNESDAY,7.5,8.5),
    (1, DayOfTheWeek.WEDNESDAY,9.5,12),
    (1, DayOfTheWeek.THURSDAY, 7,8),
    (1, DayOfTheWeek.THURSDAY, 10.5,12)
])

employee6.schedule.add_items([
    (1, DayOfTheWeek.TUESDAY,7,8),
    (1, DayOfTheWeek.TUESDAY,9,9.5),
    (1, DayOfTheWeek.TUESDAY,10,11),
    (1, DayOfTheWeek.WEDNESDAY,7,8.5),
    (1, DayOfTheWeek.WEDNESDAY,9.5,11),
    (1, DayOfTheWeek.FRIDAY,7,8.5),
    (1, DayOfTheWeek.FRIDAY,9,9.5),
    (1, DayOfTheWeek.FRIDAY,9.5,11),
])

employee7.schedule.add_items([
    (1, DayOfTheWeek.MONDAY, 7,8.5),
    (1, DayOfTheWeek.MONDAY, 9.5,12),
    (1, DayOfTheWeek.MONDAY, 12.5,14),
    (1, DayOfTheWeek.TUESDAY, 7,8.5),
    (1, DayOfTheWeek.TUESDAY, 9.5,12),
    (1, DayOfTheWeek.THURSDAY,7,8.5),
    (1, DayOfTheWeek.THURSDAY,9.5,12),
    (1, DayOfTheWeek.FRIDAY, 7,9.5),
])

employee8.schedule.add_items([
    (1, DayOfTheWeek.MONDAY, 9.5,12),
    (1, DayOfTheWeek.TUESDAY, 7.5,8.5),
    (1, DayOfTheWeek.TUESDAY, 9.5,10.5),
    (1, DayOfTheWeek.WEDNESDAY, 7,7.5),
    (1, DayOfTheWeek.WEDNESDAY,7.5,8.5),
    (1, DayOfTheWeek.WEDNESDAY,9.5,12),
    (1, DayOfTheWeek.THURSDAY, 7,8),
    (1, DayOfTheWeek.THURSDAY, 10.5,12)
])

employee9.schedule.add_items([
    (1, DayOfTheWeek.TUESDAY,7,8),
    (1, DayOfTheWeek.TUESDAY,9,9.5),
    (1, DayOfTheWeek.TUESDAY,10,11),
    (1, DayOfTheWeek.WEDNESDAY,7,8.5),
    (1, DayOfTheWeek.WEDNESDAY,9.5,11),
    (1, DayOfTheWeek.FRIDAY,7,8.5),
    (1, DayOfTheWeek.FRIDAY,9,9.5),
    (1, DayOfTheWeek.FRIDAY,9.5,11),
])

employee10.schedule.add_items([
    (1, DayOfTheWeek.MONDAY, 7,8.5),
    (1, DayOfTheWeek.MONDAY, 9.5,12),
    (1, DayOfTheWeek.MONDAY, 12.5,14),
    (1, DayOfTheWeek.TUESDAY, 7,8.5),
    (1, DayOfTheWeek.TUESDAY, 9.5,12),
    (1, DayOfTheWeek.THURSDAY,7,8.5),
    (1, DayOfTheWeek.THURSDAY,9.5,12),
    (1, DayOfTheWeek.FRIDAY, 7,9.5),
])

employees = [employee1, employee2, employee3, employee4, employee5, employee6, employee7, employee8, employee9, employee10]


In [28]:
# define dummy patients
patient1 = Patient(1)
patient2 = Patient(2)
patient3 = Patient(3)
patient4 = Patient(4)
patient5 = Patient(5)
patient6 = Patient(6)
patient7 = Patient(7)
patient8 = Patient(8)
patient9 = Patient(9)
patient10 = Patient(10)

patient1.schedule.add_items([
        (1, DayOfTheWeek.MONDAY, 9.5,10.5),
        (1, DayOfTheWeek.MONDAY, 11,11.5),
        (1, DayOfTheWeek.FRIDAY, 7.5,9.5),
    ])

patient2.schedule.add_items([
    (1, DayOfTheWeek.TUESDAY,7,8),
    (1, DayOfTheWeek.FRIDAY,7.5,9.5),
    ])

patient3.schedule.add_items([
    (1, DayOfTheWeek.WEDNESDAY,7,8.5),
    (1, DayOfTheWeek.WEDNESDAY,9.5,11),
    (1, DayOfTheWeek.THURSDAY,10,11.5),
])

patient4.schedule.add_items([
    (1, DayOfTheWeek.MONDAY, 9.5,10.5),
    (1, DayOfTheWeek.MONDAY, 11,11.5),
    (1, DayOfTheWeek.FRIDAY, 7.5,9.5),
    ])

patient5.schedule.add_items([
    (1, DayOfTheWeek.TUESDAY,7,8),
    (1, DayOfTheWeek.FRIDAY,7.5,9.5),
    ])

patient6.schedule.add_items([
    (1, DayOfTheWeek.WEDNESDAY,7,8.5),
    (1, DayOfTheWeek.WEDNESDAY,9.5,11),
    (1, DayOfTheWeek.THURSDAY,10,11.5),
])

patient7.schedule.add_items([
    (1, DayOfTheWeek.MONDAY, 9.5,10.5),
    (1, DayOfTheWeek.MONDAY, 11,11.5),
    (1, DayOfTheWeek.FRIDAY, 7.5,9.5),
    ])

patient8.schedule.add_items([
    (1, DayOfTheWeek.TUESDAY,7,8),
    (1, DayOfTheWeek.FRIDAY,7.5,9.5),
    ])

patient9.schedule.add_items([
    (1, DayOfTheWeek.WEDNESDAY,7,8.5),
    (1, DayOfTheWeek.WEDNESDAY,9.5,11),
    (1, DayOfTheWeek.THURSDAY,10,11.5),
])

patient10.schedule.add_items([
    (1, DayOfTheWeek.MONDAY, 9.5,10.5),
    (1, DayOfTheWeek.MONDAY, 11,11.5),
    (1, DayOfTheWeek.FRIDAY, 7.5,9.5),
])


patients = [patient1, patient2, patient3]

In [27]:
ct = list(itertools.combinations((employees), 2))
ct.extend(employees)
print(ct)

[(<domain.entities.Employee object at 0x0000026E873E8710>, <domain.entities.Employee object at 0x0000026E873E8850>), (<domain.entities.Employee object at 0x0000026E873E8710>, <domain.entities.Employee object at 0x0000026E873E8BD0>), (<domain.entities.Employee object at 0x0000026E873E8710>, <domain.entities.Employee object at 0x0000026E873E8C90>), (<domain.entities.Employee object at 0x0000026E873E8710>, <domain.entities.Employee object at 0x0000026E873E8D50>), (<domain.entities.Employee object at 0x0000026E873E8710>, <domain.entities.Employee object at 0x0000026E873E8E10>), (<domain.entities.Employee object at 0x0000026E873E8710>, <domain.entities.Employee object at 0x0000026E873E8F10>), (<domain.entities.Employee object at 0x0000026E873E8710>, <domain.entities.Employee object at 0x0000026E873E8FD0>), (<domain.entities.Employee object at 0x0000026E873E8710>, <domain.entities.Employee object at 0x0000026E873E90D0>), (<domain.entities.Employee object at 0x0000026E873E8710>, <domain.entit

In [29]:
# Check if a therapist can participate at the same groupsession
ct_check = []
for therapist_combo in ct:
    if type(therapist_combo) == tuple:
        if therapist_combo[0].schedule.get_numeric_template() == therapist_combo[1].schedule.get_numeric_template():
                if therapist_combo not in ct_check:
                    ct_check.append(therapist_combo)
    else:
        ct_check.append(therapist_combo)
print(ct_check)

[(<domain.entities.Employee object at 0x0000026E873E8710>, <domain.entities.Employee object at 0x0000026E873E8C90>), (<domain.entities.Employee object at 0x0000026E873E8710>, <domain.entities.Employee object at 0x0000026E873E8F10>), (<domain.entities.Employee object at 0x0000026E873E8710>, <domain.entities.Employee object at 0x0000026E873E9210>), (<domain.entities.Employee object at 0x0000026E873E8850>, <domain.entities.Employee object at 0x0000026E873E8D50>), (<domain.entities.Employee object at 0x0000026E873E8850>, <domain.entities.Employee object at 0x0000026E873E8FD0>), (<domain.entities.Employee object at 0x0000026E873E8BD0>, <domain.entities.Employee object at 0x0000026E873E8E10>), (<domain.entities.Employee object at 0x0000026E873E8BD0>, <domain.entities.Employee object at 0x0000026E873E90D0>), (<domain.entities.Employee object at 0x0000026E873E8C90>, <domain.entities.Employee object at 0x0000026E873E8F10>), (<domain.entities.Employee object at 0x0000026E873E8C90>, <domain.entit

In [30]:
cp = list(itertools.combinations((patients), 8 ))
cp.extend(itertools.combinations((patients), 7 ))
cp.extend(itertools.combinations((patients), 6 ))
cp.extend(itertools.combinations((patients), 5 ))
cp.extend(itertools.combinations((patients), 4 ))
cp.extend(itertools.combinations((patients), 3 ))
cp.extend(itertools.combinations((patients), 2 ))
cp.extend(patients)
print(cp)

[(<domain.entities.Patient object at 0x0000026E872AB690>, <domain.entities.Patient object at 0x0000026E872B84D0>, <domain.entities.Patient object at 0x0000026E872B8A50>), (<domain.entities.Patient object at 0x0000026E872AB690>, <domain.entities.Patient object at 0x0000026E872B84D0>), (<domain.entities.Patient object at 0x0000026E872AB690>, <domain.entities.Patient object at 0x0000026E872B8A50>), (<domain.entities.Patient object at 0x0000026E872B84D0>, <domain.entities.Patient object at 0x0000026E872B8A50>), <domain.entities.Patient object at 0x0000026E872AB690>, <domain.entities.Patient object at 0x0000026E872B84D0>, <domain.entities.Patient object at 0x0000026E872B8A50>]


In [32]:
# Check if a patient can participate at the same groupsession
cp_check = []
a = []
for patient_combo in cp:
    if type(patient_combo) == tuple:
        for patient in patient_combo:
            patient_schedule = patient.schedule.get_numeric_template()
            a.append(patient_schedule)

        f = a[0]
        chk = True
        for item in a:
            if f != item:
                chk = False
                break
        if chk == True:
            cp_check.append(patient_combo)
        else:
            continue
    else:
        cp_check.append(patient_combo)
print(cp_check)

[<domain.entities.Patient object at 0x0000026E872AB690>, <domain.entities.Patient object at 0x0000026E872B84D0>, <domain.entities.Patient object at 0x0000026E872B8A50>]


In [12]:
# Desicion variables for each combination of therapist and patient
x = LpVariable.dicts("x", (ct_check, cp_check), lowBound=0, upBound=1, cat=plp.LpInteger)

In [13]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem("Therapist Patient Assignment", LpMaximize)

c:\Users\Tibo.Vanbrabandt\AppData\Local\Programs\Python\Python311\Lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [14]:
# Objective function - maximize the number of patients assigned to therapists
prob += lpSum([x[therapist_combo][patient_combo] for therapist_combo in ct_check for patient_combo in cp_check])

In [25]:
# Constraint assign each patient to a therapist or assign multiple patients to a group of therapists
for patient_combo in cp_check:
    prob += lpSum([x[therapist_combo][patient_combo] for therapist_combo in ct_check]) == 1
    print(lpSum([x[therapist_combo][patient_combo] for therapist_combo in ct_check]))
    print(" ")

x_Schedule_template_for_employee_1
MONDAY_from_7_to_8.5
MONDAY_from_9.5_to_12
MONDAY_from_12.5_to_14
TUESDAY_from_7_to_8.5
TUESDAY_from_9.5_to_12
THURSDAY_from_7_to_8.5
THURSDAY_from_9.5_to_12
FRIDAY_from_7_to_9.5_Schedule_template_for_patient_1
MONDAY_from_9.5_to_10.5
MONDAY_from_11_to_11.5
FRIDAY_from_7.5_to_9.5 + x_Schedule_template_for_employee_2
MONDAY_from_9.5_to_12
TUESDAY_from_7.5_to_8.5
TUESDAY_from_9.5_to_10.5
WEDNESDAY_from_7_to_7.5
WEDNESDAY_from_7.5_to_8.5
WEDNESDAY_from_9.5_to_12
THURSDAY_from_7_to_8
THURSDAY_from_10.5_to_12_Schedule_template_for_patient_1
MONDAY_from_9.5_to_10.5
MONDAY_from_11_to_11.5
FRIDAY_from_7.5_to_9.5 + x_Schedule_template_for_employee_3
TUESDAY_from_7_to_8
TUESDAY_from_9_to_9.5
TUESDAY_from_10_to_11
WEDNESDAY_from_7_to_8.5
WEDNESDAY_from_9.5_to_11
FRIDAY_from_7_to_8.5
FRIDAY_from_9_to_9.5
FRIDAY_from_9.5_to_11_Schedule_template_for_patient_1
MONDAY_from_9.5_to_10.5
MONDAY_from_11_to_11.5
FRIDAY_from_7.5_to_9.5
 
x_Schedule_template_for_employee_1


In [21]:
# Print schedule for therapist